In [138]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [139]:
df = pd.read_excel("extrem.xlsx")

In [140]:
df.head()

,Datetime,Unnamed: 1,temp,Unnamed: 3,tnz,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,tt,t_tenzo,tnzl,dif
0,2024-03-18 14:06:40,8,34.6248,26.800,2183140,2178584,2178557,0,0,42.61,36.4642,26,52.15,0,-163,35.108667,2183140,0
1,2024-03-18 14:06:52,8,35.1197,24.120,2183170,2178549,2178525,0,0,38.31,36.9437,24,52.47,0,-861,35.574000,2183170,30
2,2024-03-18 14:08:05,8,35.5940,21.708,2183159,2178476,2178503,0,0,37.17,37.4262,-27,52.73,0,-1603,36.068667,2183159,-11
3,2024-03-18 14:09:17,8,36.0510,19.537,2183152,2178408,2178484,0,0,37.02,37.8699,-75,52.96,0,-2266,36.510667,2183152,-7
4,2024-03-18 14:10:30,8,36.4945,17.584,2183170,2178368,2178466,0,0,37.15,38.3060,-98,53.17,0,-2935,36.956667,2183170,18


In [141]:
desired_columns = ['Datetime','t_tenzo','tnzl']

In [142]:
df = df.drop(df.columns.difference(desired_columns),axis=1)

In [143]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Datetime  909 non-null    datetime64[ns]
 1   t_tenzo   909 non-null    float64       
 2   tnzl      909 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 21.4 KB


In [144]:
df.head()

,Datetime,t_tenzo,tnzl
0,2024-03-18 14:06:40,35.108667,2183140
1,2024-03-18 14:06:52,35.574000,2183170
2,2024-03-18 14:08:05,36.068667,2183159
3,2024-03-18 14:09:17,36.510667,2183152
4,2024-03-18 14:10:30,36.956667,2183170


In [145]:
def create_plot(df):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df['Datetime'], y=df['tnzl'], mode='markers+lines', name='tenzo'))
    fig.add_trace(go.Scatter(x=df['Datetime'], y=df['t_tenzo'], mode='markers+lines', name='Temp', yaxis='y2'))

    fig.update_layout(
        title='График Temp и tenzo от времени',
        title_x=0.5,
        xaxis=dict(title='Datetime'),
        yaxis=dict(tickformat ='d',title='Tenzo', side='left', visible=True),  
        yaxis2=dict(tickformat ='d',title='Temp', overlaying='y', side='right', visible=True),

        hovermode="x",
        font=dict(size=14),
        margin=dict(l=100, r=100, t=50, b=50) 
    )

    fig.show()

In [146]:
create_plot(df)

In [147]:
pearson_corr = df['t_tenzo'].corr(df['tnzl'], method='pearson')

# Посчитаем коэффициент Спирмена
spearman_corr = df['t_tenzo'].corr(df['tnzl'], method='spearman')

print("Коэффициент Пирсона:", pearson_corr)
print("Коэффициент Спирмена:", spearman_corr)

Коэффициент Пирсона: 0.8247921135254261
Коэффициент Спирмена: 0.87142001400131


In [148]:

df['Datetime'] = pd.to_datetime(df['Datetime'])
df = df.sort_values(by='Datetime')
df['temp_diff'] = df['t_tenzo'].diff()

print(df[['Datetime', 't_tenzo', 'temp_diff']])

               Datetime    t_tenzo  temp_diff
0   2024-03-18 14:06:40  35.108667        NaN
1   2024-03-18 14:06:52  35.574000   0.465333
2   2024-03-18 14:08:05  36.068667   0.494667
3   2024-03-18 14:09:17  36.510667   0.442000
4   2024-03-18 14:10:30  36.956667   0.446000
..                  ...        ...        ...
904 2024-03-19 08:22:47  31.804000  -0.012000
905 2024-03-19 08:24:00  31.790667  -0.013333
906 2024-03-19 08:25:12  31.778000  -0.012667
907 2024-03-19 08:26:24  31.755333  -0.022667
908 2024-03-19 08:27:37  31.738000  -0.017333

[909 rows x 3 columns]


In [151]:
fig_scatter = px.scatter(df, x='temp_diff', y='tnzl', title='Диаграмма рассеяния Temp и tenzo')
fig_scatter.update_yaxes(tickformat='d')
fig_scatter.show()

In [166]:
diff_tenzo = df.tnzl + (df.temp_diff+df.t_tenzo)**2 * 0.38 -(df.temp_diff+df.t_tenzo) * 70.968

In [167]:
fig = go.Figure()

fig.add_trace(go.Scatter(y=df['tnzl'], mode='markers+lines', name='tenzo'))
fig.add_trace(go.Scatter(y=diff_tenzo, mode='markers+lines', name='Temp', yaxis='y2'))

fig.update_layout(
    title='График Temp и tenzo от времени',
    title_x=0.5,
    xaxis=dict(title='Datetime'),
    yaxis=dict(tickformat ='d',title='Tenzo', side='left', visible=True),  
    yaxis2=dict(tickformat ='d',title='Temp', overlaying='y', side='right', visible=True),

    hovermode="x",
    font=dict(size=14),
    margin=dict(l=100, r=100, t=50, b=50) 
)

fig.show()

In [ ]:
# Создание матрицы признаков X (температура и вес)
X = df[['t_tenzo']]

# Создание целевой переменной y (тензосигнал)
y = df['tnzl']

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)

# Создание и обучение модели линейной регрессии
model = LinearRegression()
model.fit(X_train, y_train)

# Предсказание значений для тестового набора
y_pred = model.predict(X_test)

# Оценка модели с помощью среднеквадратичной ошибки
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Вывод коэффициентов регрессии
print("Коэффициенты регрессии:", model.coef_)

# Вывод свободного члена
print("Свободный член:", model.intercept_)

Mean Squared Error: 39630.45968469198
Коэффициенты регрессии: [39.1279449]
Свободный член: 2182592.835015194
